In [166]:
from html.parser import HTMLParser
from os.path import basename
import re
from urllib.parse import quote_plus, quote
from nbformat import read
from nbconvert import MarkdownExporter
from nbconvert.writers import FilesWriter
from nbconvert.nbconvertapp import NbConvertApp
from nbconvert.preprocessors import Preprocessor
from bs4 import BeautifulSoup

In [67]:
notebook = './1-MonteCarlo-Resource-Assessments-part1.ipynb'

In [201]:
with open(notebook, 'r', encoding='utf-8') as f:
            nb = read(f, 4)
        
try:
    output = quote_plus(
        nb.metadata.date + '-' + nb.metadata.title, safe='/'
    )
except:
    output = ''

In [202]:
# class MyHTMLParser(HTMLParser):
    
#     def handle_starttag(self, tag, attrs):
#         if tag == 'img':
#             for a in attrs:
#                 if a[0] == 'src':
#                     files.append(a[1])


In [299]:
class CustomPreprocess(Preprocessor):

    
    def preprocess(self, nb, resources):
        
        global files
        files = []

        pre = """<div class="equation">\n\t<div>"""
        post = """</div>\n<div class="equation_dots"></div>\n"""
        post += "<div></div>\n"
        post += "</div>\n"
        
        img_pre = '{{ site.url }}' + 'assets/images/posts/' + quote(output) 
        
        for cell in nb.cells:
            if cell.cell_type == 'markdown':
                s = re.sub("^\# .+\n\n", '', cell.source, re.MULTILINE)
                s = re.sub(r'\\\\\[', pre + r'\\[', s)
                s = re.sub(r'\\\\\]', r'\\]' + post, s)
                
                
                soup = BeautifulSoup(s, 'html.parser')
                for img in soup.findAll('img'):
                    files.append(img['src'])
                    img['src'] = img_pre + basename(img['src'])
                    del img['width']
                    img['class'] = 'scaled'
                
                
                for caption in soup.findAll('figcaption'):
                    c = caption.get_text()
                    c = re.sub('^Figure \d+: ', '', c)
                    caption.string = c
                    print(c)
                    
                s = str(soup)
                
                
                cell.source = s
                
                
        try:
            nb.metadata.notebook
        except:
            nb.metadata.notebook = output
            
        return nb, resources

In [300]:
md_exporter = MarkdownExporter(
    template_name="blog-markdown",
    preprocessors=[CustomPreprocess]
)

In [301]:
md_writer = FilesWriter(
    build_directory = "/Users/dom/Downloads/nbconvert_output",
)


In [302]:
app = NbConvertApp(
    output_base = output
)
app.exporter = md_exporter
app.writer = md_writer

In [303]:
md_writer.files = files
app.convert_single_notebook(notebook)

[NbConvertApp] Converting notebook ./1-MonteCarlo-Resource-Assessments-part1.ipynb to 


Figure title.


In [278]:
files

['./assets/1-sensitivity.png']

In [165]:
urlparse(output)

ParseResult(scheme='', netloc='', path='2021-07-27-Monte+Carlo+resource+assessments+in+Python', params='', query='', fragment='')